<a name="top">
    Top of page
</a>

## 5B - Hands on Building of a Geographic Model
### Facility Location Problems - Demonstration

#### Case Study: Sexual Health Clinics in Hampshire

The data provided below is taken from a real facility location study conducted in Hampshire. Data scientists worked with commissioners, a public health team, and a community NHS trust to review the provision of sexual health clinics across the region.

There were concerns about the current provision of sexual health care. One of these concerns related to sustainability of services across **28** different locations. Furthermore, the variety in care provided at different locations was also of concern. The data scientists were tasked with identifying promising configurations of clinics to be retained, that preserved fair access for patients.

**By the end of this exercise you'll have been shown how to:**

* represent a facility location problem in a general format suitable for algorithmic solution
* solve the p-median facility location problem using a brute force approach
* create a chart to compare and contrast potential solutions.

The following sections of code are included within this notebook:

[1. Library Imports](#lib_imports)\
[2. Data Imports](#data_imports)\
[3. Representing a Solution](#rep_sol)\
[4. Constructing a Random Solution](#cons_random)\
[5. Evaluating a Solution](#eval_sol)\
[6. Small Problem: Enumerating all Possible Combinations](#small_prob)\
[7. Bruteforce Solution](#bruteforce)\
[8. Graphical Representation of Bruteforce Solution](#bar_chart)\
[9. Medium to Large: Using random restarts](#med_large)

**Note** Using line numbers within cells may aid readability. Press `Shift` + `L` to toggle on/ off.

### Library Imports
<a name="lib_imports">
   Code to Import Libraries
</a>

In [ ]:
# Standard Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# combinations from the itertools library allows us to enumerate all 
# solutions (for small instances).
from itertools import combinations

In [ ]:
# Metapy package imports
# Required for a Weighted Average Objective
# Ensure you have the 'metapy' folder (and contents) within your 
# current working directory
from metapy.evolutionary.evolutionary import WeightedAverageObjective

### Data Imports
<a name="data_imports">
   Code to Import Data
</a>

In [ ]:
# Travel times to each clinic from different locations
travel_matrix = pd.read_csv('../data/clinic_car_travel_time.csv', 
                            index_col='sector')
travel_matrix.head()

In [ ]:
# no of cases by postcode sector...
cases = pd.read_csv('../data/sh_demand.csv', index_col='sector')
cases.head()

### Representing a solution
<a name="rep_sol">
   Code to Represent a Proposed Solution
</a>

When tasked with a facility location problem, its important think about how any proposed solution(s) will be represented. In this case, the solution will be represented using a vector of length $p$ where $p <= P$.  

Each element of the vector represents the index of a clinic.  For example, if we have $P = 28$ candidate locations for clinics and wish to find the best solution where $p = 4$

```python
solution = np.array([10, 0, 6, 12])
```
The above code means that clinics which indicies 10, 0, 6, and 12 are included in the solution.

To select a set of clinics from a `pandas.DataFrame` you can use the code below.  The indicies are used to select the column headers. In turn the column headers are then used to select the column data from the dataframe.

In [ ]:
# the solution represents the indicies of clinics in the travel matrix
solution = np.array([10, 0, 6, 12])

# this code looks up the column names using the indicies in the solution
# if you are unsure what it does then print out `mask`
mask = travel_matrix.columns[solution]

# finally we select a restricted set of columns from the DataFrame
travel_matrix[mask]

In [ ]:
print(mask)

### Constructing a Random Solution
<a name="cons_random">
   Code to Represent a Proposed Solution
</a>

Usually an initial solution(s) to a facility location problem is **generated** (as opposed to being manually specified). This gives you starting place by which to assess other solutions (i.e., better or worse than the one(s) before it).

The function below `random_solution` accepts:
* `n_candidates`: The number of candidate locations where you could place clinics (facilities)
* `p` the number of clinics to place.
* `random_seed` (optional).  Use if you wanted to recreate your results.

In [ ]:
def random_solution(n_candidates, p, random_seed=None):
    '''
    Helper function to generate a random solution
    
    Params
    ------
    n_candidates : int
        The number of candidate locations where you could place 
        clinics (facilities).
        
    p : int
        The number of clinics to place.
        
    random_seed : int (Default=None)
        Random seed for reproducibility.
    
    Returns
    -------
    
    Vector (np.array) of length p
    '''
    # create a random number generator
    rng = np.random.default_rng(seed=random_seed)

    # sample without replacement
    solution = []
    while len(solution) < p:
        candidate = rng.integers(0, n_candidates)
        if candidate not in solution:
            solution.append(candidate)
            
    return np.array(solution)

In [ ]:
# Now we'll use that fn to generate a random solution
# This will then be saved to the variable `init_solution`
init_solution = random_solution(28, 4, random_seed=42)
init_solution

**Question:** What would happen if we removed the random seed parameter and executed the function 10 times?

**Advanced** Although not necessary, if we wanted to refactor the code and make it more elegant, Numpy's built-in `choice` method that allows you to sample **without** replacement.

[`np.arange` documentation](https://numpy.org/doc/stable/reference/generated/numpy.arange.html)

In [ ]:
def random_solution2(n_candidates, p, random_seed=None):
    '''
    Helper function to generate a random solution
    
    Params
    ------
    n_candidates : int
        The number of candidate locations where you could place clinics (facilities).
        
    p : int
        The number of clinics to place.
        
    random_seed : int (Default=None)
        Random seed for reproducibility.
    
    Returns
    -------
    
    Vector (np.array) of length p
    '''
    # create a random number generator
    rng = np.random.default_rng(seed=random_seed)
    
    # create array of candidate indices
    candidates = np.arange(n_candidates, dtype=np.byte)
    
    # sample without replacement and return array
    return rng.choice(candidates, size=p, replace=False)

In [ ]:
init_solution = random_solution2(28, 4)
init_solution

### Evaluating a Solution
<a name="eval_sol">
   Code to Represent a Proposed Solution
</a>

There are multiple ways to formulate the objectives of a facility location problem.  With the sexual health case study you will formulate it as a p-median problem.  Where the objective is to find the combination of facilities that minimises the weighted average car travel time to a clinic.

`metapy` contains a `WeightAverageObjective` class that accepts both the geospatial demand and travel matrix as parameters.

```python
from metapy.evolutionary.evolutionary import WeightedAverageObjective

# create an instance and pass in demand and travel times
obj = WeightedAverageObjective(cases, travel_matrix)
```

The object has a method `evaluate(solution)` that accepts a numpy vector that is your representation of the a solution.

```python
obj.evaluate(solution)
```

The code below brings these together.  Execute the cell to see the output.

[Back to the top of this page.](#top)

In [ ]:
obj = WeightedAverageObjective(cases, travel_matrix)
# `solution` was manually created above...
obj.evaluate(solution)

In [ ]:
# compare the the initial solution
obj.evaluate(init_solution)

#### Important Note

In this example, we've been provided with pre-aggregated demand per 'sector' (i.e. each sector is listed once and the cumulative number of corresponding visits (i.e., demand) is provided. However, if the data had been provided in a non-aggregated form i.e., granular list of each unique visit, then it would be possible to calculate both the maximum and 95th percentile of travel times, per clinic.


... More detail to be added to aid the Group Exercise

### Small Problem: Enumerating all Possible Combinations
<a name="small_prob">
   Code to Enumerate all Possible Facility Combinations.
</a>

For small problems it is possible to enumerate all combinations to locate the "optimal" solution.  

The function `all_combinations` below will provide a list of solutions representing an exhaustive solution space for a given problem.  For example, if the problem consisted of 10 candidate locations and you wish to evaluate solutions of size 4 then the solution space is fully represented by 210 unique solutions.

This method is straightforward for health service customers to understand and for small problems it is unusual to use a complex optimisation procedure.  As the facility location problem is NP hard, this strategy is not recommended for large scale problems.  The evaluation of the fitness of the solutions will become prohibitive as the size of the problem begins to exceed 10.

**Question** What is a list comprehension? [Reminder here](https://www.w3schools.com/python/python_lists_comprehension.asp).

In [ ]:
def all_combinations(n_facilities, p):
    '''
    n_facilities : int
        The number of candidate locations where you could place facilities (clinics).
        
    p : int
        The number of clinics to place.
    
    Returns
    -------
    
    Returns all p sized combinations of an array containing
    indicies 0 to n_facilties - 1 
    '''
    facility = np.arange(n_facilities, dtype=np.uint8)
    return [np.array(a) for a in combinations(facility, p)]

In [ ]:
# size 4 combinations of 10 candidate locations = 210
comb = all_combinations(n_facilities=10, p=4)
len(comb)

In [ ]:
# take a look at index 0
comb[0]

In [ ]:
# index 209 i.e., last
comb[-1]

### Bruteforce Solution
<a name="bruteforce">
   Code to assess all possible clinic combinations with respect ot weighted average travel time.
</a>

Given that there are a relatively small number of combinations available, it is possible to assess each of them before determining the optimal solution.

The steps for this are:
1. Generate all possible combinations using the `all_combinations` function;
2. Create a fresh WeightedAverageObjective varaible, using the cases and travel matrix data;
3. Evaluate each solution and append the results to a list, before converting the completed list to a Numpy array.
4. Identify the index of the optical solution (from within the array of the combinations). [See `np.argmin` documentation here](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html).

In [ ]:
# size 4 combinations of 6 candidate locations = 15 combinations
comb = all_combinations(n_facilities=6, p=4)
print('With 4 ouf of 6 possible locations, there are'+\
      f' {len(comb)} combinations')

# create objective function...
obj = WeightedAverageObjective(cases, travel_matrix)

results = []
for solution in comb:
    results.append(obj.evaluate(solution))
results = np.array(results)

# get the index of the minimum
optimal_index = np.argmin(results)
print('The index of the combinations offering the optimal'+\
      f' solution is {optimal_index}')

### Graphical Representation of Bruteforce Solution
<a name="bar_chart">
   Code to generate a bar chart representing to assessment above.
</a>

When considering all of the potential solutions (of which there are 15).

**Questions**
1. How many bars will be in this chart?
2. How do you think you could colour the bar of the optimal combination yellow?

In [ ]:
# Create axis on which to generate the chart
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# convert solutions to strings
xlabels = []
for solution in comb:
    label = '-'.join(str(a) for a in solution)
    xlabels.append(label)

barlist = ax.bar(xlabels, results)
barlist[optimal_index].set_color('r')

print(f'optimal cost: {results[optimal_index]}')
print(f'optimal solution: {comb[optimal_index]}')

*Additional actions...*
1. Can you modify the code below to now:\
    a. Add a Title to the chart?\
    b. Add an x-axis label to the chart?\
    c. Add a y-axis label to the chart?\
    b. Save the chart as an image?

In [ ]:
# Code below is currently a Copy and Paste of the above...
# Can you address the four additional actions?

# Create axis on which to generate the chart
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# convert solutions to strings
xlabels = []
for solution in comb:
    label = '-'.join(str(a) for a in solution)
    xlabels.append(label)

barlist = ax.bar(xlabels, results)
barlist[optimal_index].set_color('r')

print(f'optimal cost: {results[optimal_index]}')
print(f'optimal solution: {comb[optimal_index]}')

In [ ]:
# Code below is currently a Copy and Paste of the above...
# Can you address the four additional actions?

# Create axis on which to generate the chart
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# convert solutions to strings
xlabels = []
for solution in comb:
    label = '-'.join(str(a) for a in solution)
    xlabels.append(label)

barlist = ax.bar(xlabels, results)
barlist[optimal_index].set_color('r')

# Add Title
ax.set_title("Visual Comparison of Solutions by Clinic Index Numbers")

# Add x-axis label
ax.set_xlabel("Clinic Combinations (Index)")

# Add y-axis label
ax.set_ylabel("Travel 'Cost'")

# Save file
fig.savefig("sols_chart_ca1.png")

print(f'optimal cost: {results[optimal_index]}')
print(f'optimal solution: {comb[optimal_index]}')

### Medium to Large: Using random restarts
<a name="med_large">
   Code to use random restarts/ random search as a simple benchmark.
</a>

When you encounter a medium to large instance of a facility location problem (or any optimisation problem) it is often tempting to move quickly onto a sophisticated solution method.  But how do you know that it is better than a simple heuristic?  

**Here you will use the random restarts algorithm (sometimes called random search) as a simple benchmark.**

Random restarts is simple heuristic.  You allocate a computational budget - either in terms of iterations or execution time - and randomly generate solutions and evaluate them. For example, you might specify an execution time limit of 10 seconds and in that time evaluate as many random solutions as possible.  

In the cells below, we will demonstrate the following:
* Using the `random_solution` function defined earlier create another function `random_restarts` that runs for fixed number of iterations i.e., 10.  This will mean that the algorithm will generate 10 random solutions and then evaluates them.  
* The function returns a tuple of the best cost and best solution found when minimising the weighted average car travel time.
* The use case will be 28 candidate locations and a budget of 8 clinics to place - that's over 3 millions possible combinations.
* Try running the algorithm a few times or varying the budget (number of clinics). What do you notice?

In [ ]:
def random_restarts(max_iter, obj, n_facilities, p, random_seed=None):
    '''
    max_iter : int
        Maximum number of iterations to try
    
    obj :  object
        WeightedAverageObjective
    
    n_facilities : int
        The number of candidate locations where you could place facilities (clinics).
        
    p : int
        The number of clinics to place.
    
    Returns
    -------
    
    best_cost : float
        Lowest 'cost'    
    
    solution : array
        Indecies of clincs in solution.
    '''
    
    np.random.seed(random_seed)
    
    # implementation of random restarts alg
    best_cost = np.Inf
    best_solution = None
    for i in range(max_iter):
        solution = random_solution(n_facilities, p)
        cost = obj.evaluate(solution)
        
        if cost < best_cost:
            best_cost = cost
            best_solution = solution
            
    return best_cost, solution
        

Showing the random nature by running the code twice...

In [ ]:
# First time...
max_iter = 1000
n_facilities = 28
p = 8

# create objective function...
obj = WeightedAverageObjective(cases, travel_matrix)

random_restarts(max_iter, obj, n_facilities, p)

In [ ]:
# Second time...
max_iter = 1000
n_facilities = 28
p = 8

# create objective function...
obj = WeightedAverageObjective(cases, travel_matrix)

random_restarts(max_iter, obj, n_facilities, p)

The end.